# Phase Diagram Demo

This notebook demonstrates solving user-defined complex ODEs with multiple initial conditions, plotting trajectories, and saving all results to a timestamped folder.

Steps:
- Ensure the src directory is on sys.path for imports
- Define paths to the JSON IC file and the vanderpol system
- Run the solver via the package's run() function
- Preview generated figures

In [ ]:
# Setup import path to use src/ layout
import os, sys, pathlib
root = pathlib.Path("..").resolve()
src = root / "src"
os.environ.setdefault("PYTHONPATH", str(src))
if str(src) not in sys.path:
    sys.path.insert(0, str(src))
print("Using src path:", src)

# Optional: ensure required packages are available
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Configure inputs
ic_json = str((root / "notebooks" / "vdp_ic.json").resolve())
system_py = str((root / "src" / "phase_diagram" / "systems" / "vanderpol.py").resolve())
func_name = "vanderpol"

print("IC JSON:", ic_json)
print("System file:", system_py)
print("Function:", func_name)

In [ ]:
# Run the solver and save outputs into a timestamped folder
from phase_diagram.main import run

# Choose PSD signal interactively for this run
psd_signal_in = input("PSD signal ['amplitude' or 'intensity'] (default amplitude): ").strip().lower()
psd_signal = psd_signal_in if psd_signal_in in ('amplitude','intensity') else 'amplitude'

run_dir = run(
    ic_json=ic_json,
    system_py=system_py,
    func=func_name,
    var_index=0,
    mod_i=0,
    mod_j=0,
    base_out_dir=str(root / "runs"),
    t_points=1000,
    show=False,
    psd_signal=psd_signal,
)
print("Saved outputs to:", run_dir)

In [ ]:
# Preview generated figures
from IPython.display import Image, display
from pathlib import Path
figs_dir = Path(run_dir) / "figs"
for name in ["phase_trajectories.png", "modulus_phase_trajectories.png"]:
    p = figs_dir / name
    if p.exists():
        display(Image(filename=str(p)))
    else:
        print("Missing figure:", p)

In [ ]:
# Replot from an existing run directory
# - Provide the folder path containing metadata.json and solutions.npz
# - Optionally override acquire_interval (t0, t1)
# - Choose what to draw: phase trajectories and/or PSD
# - PSD plotting supports:
#   * axis scale (linear/semilogy/semilogx/loglog)
#   * freq_range [fmin, fmax]
#   * signal: 'amplitude' (|z|) or 'intensity' (complex z two-sided)

from pathlib import Path
import json
from phase_diagram.main import replot_run

# User inputs
run_dir_in = input("Enter run directory path (or leave empty to use last run_dir): ").strip()
if not run_dir_in:
    run_dir_in = run_dir if 'run_dir' in globals() else ''
if not run_dir_in:
    raise RuntimeError("No run directory provided and no previous run available.")

# Load metadata for defaults
meta_path = Path(run_dir_in) / 'metadata.json'
if not meta_path.exists():
    raise FileNotFoundError(f"metadata.json not found in {run_dir_in}")
with open(meta_path, 'r', encoding='utf-8') as f:
    meta = json.load(f)

plotting_meta = meta.get('plotting', {}) if isinstance(meta.get('plotting'), dict) else {}
# Defaults from metadata
acq_default = plotting_meta.get('acquire_interval')
psd_axis_default = plotting_meta.get('psd_axis_scale', 'linear')
psd_freq_default = plotting_meta.get('psd_freq_range')
psd_signal_default = plotting_meta.get('psd_signal', 'amplitude')

# Optional overrides via inputs
acq_in = input(f"Acquire interval override (t0,t1) or empty to use {acq_default}: ").strip()
if acq_in:
    parts = [p for p in acq_in.split(',') if p]
    if len(parts) >= 2:
        acquire_interval = (float(parts[0]), float(parts[1]))
    else:
        raise ValueError("acquire_interval must be 't0,t1' if provided")
else:
    acquire_interval = tuple(acq_default) if isinstance(acq_default, list) and len(acq_default) >= 2 else None

plot_phase = input("Plot phase trajectories? [y/N]: ").strip().lower() == 'y'
plot_psd = input("Plot PSD? [Y/n]: ").strip().lower() != 'n'

# PSD axis scale, freq range, and signal overrides
axis_in = input(f"PSD axis scale [linear/semilogy/semilogx/loglog] (default {psd_axis_default}): ").strip().lower()
psd_axis_scale = axis_in if axis_in in {'linear','semilogy','semilogx','loglog'} else psd_axis_default
freq_in = input(f"PSD freq range fmin,fmax or empty (default {psd_freq_default}): ").strip()
if freq_in:
    parts = [p for p in freq_in.split(',') if p]
    if len(parts) >= 2:
        fmin, fmax = float(parts[0]), float(parts[1])
        psd_freq_range = (min(fmin,fmax), max(fmin,fmax))
    else:
        psd_freq_range = None
else:
    psd_freq_range = tuple(psd_freq_default) if isinstance(psd_freq_default, list) and len(psd_freq_default) >= 2 else None

sig_in = input(f"PSD signal ['amplitude' (|z|) or 'intensity' (complex z)] (default {psd_signal_default}): ").strip().lower()
psd_signal = sig_in if sig_in in {'amplitude','intensity'} else psd_signal_default

# Perform replot; replot_run uses metadata for PSD method/params
# Temporarily update metadata to pass plotting overrides through, then restore
backup = meta.copy()
plotting_meta2 = {**plotting_meta,
                  'psd_axis_scale': psd_axis_scale,
                  'psd_freq_range': list(psd_freq_range) if psd_freq_range else None,
                  'psd_signal': psd_signal}
meta['plotting'] = plotting_meta2
with open(meta_path, 'w', encoding='utf-8') as f:
    json.dump(meta, f, indent=2)

try:
    figs_dir = replot_run(run_dir_in, acquire_interval=acquire_interval, plot_phase=plot_phase, plot_psd=plot_psd, show=False)
    print('Replotted figures at:', figs_dir)
finally:
    # Restore original metadata
    with open(meta_path, 'w', encoding='utf-8') as f:
        json.dump(backup, f, indent=2)
